In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64)

In [3]:
class RFNet(nn.Module):
    def __init__(self):
        super(RFNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64*6*6, 64)    # 크기조정
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x,1) # 1차원으로 펼치기
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
def train_and_validate(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(5):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            output = model(inputs)
            loss = criterion(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss+=loss.item()
            if i%200 == 199:
                print('[%d, %5d] loss: %.3f' %(epoch+1, i+1, running_loss/200))
                running_loss = 0.0
            
    print('finish_training')

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _,predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()

    accuracy = 100 * correct/total
    print(f'accuracy: {accuracy:.2f}')

In [5]:
rf_model = RFNet()
train_and_validate(rf_model)

print('Num of param')
print('RFNet', sum(p.numel() for p in rf_model.parameters()))

[1,   200] loss: 2.290
[1,   400] loss: 2.208
[1,   600] loss: 2.036
[2,   200] loss: 1.834
[2,   400] loss: 1.788
[2,   600] loss: 1.742
[3,   200] loss: 1.642
[3,   400] loss: 1.603
[3,   600] loss: 1.557
[4,   200] loss: 1.482
[4,   400] loss: 1.461
[4,   600] loss: 1.444
[5,   200] loss: 1.396
[5,   400] loss: 1.372
[5,   600] loss: 1.349
finish_training
accuracy: 53.44
Num of param
RFNet 167562
